In [1]:
import conllu
import json
import csv
import os
import pandas as pd

In [2]:
def parse_conllu(text):
    try: 
        result = [tokenlist for tokenlist in conllu.parse(text)]
        return result[0]
    except:
        return None ## сюда можно подставить свой собственный разбор

In [3]:
def read_conllu_data(filename):
    with open(filename, 'r', encoding='UTF-8') as f:  
        flag = False
        result_sent_list = list()
        temp_list = list()

        while True: 
            line = f.readline()
            if not line:  # достигли конца файла
                parse_result = parse_conllu("".join(temp_list))
                if parse_result is not None:
                    result_sent_list.append(parse_result)
                return result_sent_list
            if line[:6] == "# text":
                if flag is False:  # начинаем считывать новое предложение
                    flag = True
                else:  # достигли конца разбора предложения
                    flag = False
                    parse_result = parse_conllu("".join(temp_list))
                    if parse_result is not None:
                        result_sent_list.append(parse_result)
                    temp_list = list()
            temp_list.append(line)

In [4]:
class VerbGovernment:
    """Глагольное управление: глагол/деепричастие/причастие + (предлог) + существительное; падеж"""
    
    def __init__(self, sent, noun, verb):
        self.noun = noun
        self.verb = verb
        self.part = self.__find_adp_or_part__(sent, verb['id'], 'PART')
        self.adp = self.__find_adp_or_part__(sent, noun['id'], 'ADP')
        self.case = self.__find_case__(noun)
        
    def __find_adp_or_part__(self, sent, head_id, pos):
        for i in range(head_id - 1, -1, -1):
            if sent[i]['upos'] == pos and int(sent[i]['head']) == int(head_id):
                if pos == 'PART' and sent[i]['lemma'] == 'не':
                    return sent[i]['lemma']
                elif pos == 'ADP':
                    return sent[i]['lemma']
        return '_'
    
    def __find_case__(self, token):
        try:
            return token['feats']['Case']
        except:
            return '_'
        
    def show(self):
        print(self.part, self.verb['form'], self.adp, self.noun['form'], self.case)
        
    def dict_of_forms(self):
        res_dict = {
            'part': self.part,
            'verb': self.verb['form'].lower(),
            'adp': self.adp,
            'noun': self.noun['form'].lower(),
            'case': self.case
        }
        return res_dict
    
    def dict_of_lemmas(self):
        res_dict = {
            'part': self.part,
            'verb': self.verb['lemma'],
            'adp': self.adp,
            'noun': self.noun['lemma'],
            'case': self.case
        }
        return res_dict

In [5]:
class Corpus:
    
    def __init__(self, filename):
        self.filename = filename
        self.dict_of_lemmas = dict()
        self.list_of_forms = list()
        self.count_parsed = 0
        self.count_error = 0
        
        self.__read_conllu_data__(filename)
        self.stat = self.__calculate__()
        
    def __parse_conllu__(self, text):
        try: 
            result = [tokenlist for tokenlist in conllu.parse(text)]
            return result[0]
        except:
            return None ## сюда можно подставить свой собственный разбор
        
    def __find_verb_government__(self, sent):
        for token in sent:
            if token['upos'] == 'NOUN' and sent[token['head'] - 1]['upos'] == 'VERB':
                vg = VerbGovernment(sent, token, sent[token['head'] - 1])
                self.list_of_forms.append(vg.dict_of_forms())
                self.__update_dict_of_lemmas__(vg.dict_of_lemmas())
                
    def __update_dict_of_lemmas__(self, vg):
        if vg['part'] not in self.dict_of_lemmas:
            self.dict_of_lemmas[vg['part']] = dict()
        if vg['verb'] not in self.dict_of_lemmas[vg['part']]:
            self.dict_of_lemmas[vg['part']][vg['verb']] = dict()
        if vg['adp'] not in self.dict_of_lemmas[vg['part']][vg['verb']]:
            self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']] = dict()
        if vg['case'] not in self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']]:
            self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']][vg['case']] = dict()
        if vg['noun'] not in self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']][vg['case']]:
            self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']][vg['case']][vg['noun']] = 1
        else:
            self.dict_of_lemmas[vg['part']][vg['verb']][vg['adp']][vg['case']][vg['noun']] += 1
        
    def __read_conllu_data__(self, filename):
        with open(filename, 'r', encoding='UTF-8') as f:  
            flag = False
            temp_list = list()

            while True: 
                line = f.readline()
                if not line:  # достигли конца файла
                    parse_result = self.__parse_conllu__("".join(temp_list))
                    if parse_result is not None:
                        self.__find_verb_government__(parse_result)
                        self.count_parsed += 1
                    else:
                        self.count_error += 1
                    return 0
                if line[:6] == "# text":
                    if flag is False:  # начинаем считывать новое предложение
                        flag = True
                    else:  # достигли конца разбора предложения
                        flag = False
                        parse_result = self.__parse_conllu__("".join(temp_list))
                        if parse_result is not None:
                            self.__find_verb_government__(parse_result)
                            self.count_parsed += 1
                        else:
                            self.count_error += 1
                        temp_list = list()
                temp_list.append(line)
                
    def __count_verb__(self, part):
        try:
            count = len(self.dict_of_lemmas[part])
        except:
            count = 0
        return count
                
    def __calculate__(self):       
        result_dict = {
            "corpus_name": self.filename.split('/')[-1],
            "count_sent" : self.count_parsed + self.count_error,
            "count_parsed_sent": self.count_parsed, 
            "count_error_sent": self.count_error,
            "count_vg": len(self.list_of_forms), 
            "verb_": self.__count_verb__("_"), 
            "verb_not": self.__count_verb__("не") 
        }
        return result_dict
    
    def __new_filename__(self, ext):
        parts = self.filename.split('conllu')
        new_filename = parts[0][:-1] + parts[1] + ext
        return new_filename
        
    def save_dict_to_json(self, json_filename=None):
        if json_filename is None:
            json_filename = self.__new_filename__('json')
        
        with open(json_filename, 'w', encoding='UTF-8') as f:
            json.dump(self.dict_of_lemmas, f)
    
    def save_list_to_csv(self, csv_filename=None):
        if csv_filename is None:
            csv_filename = self.__new_filename__('csv')
            
        keys = self.list_of_forms[0].keys()
        
        with open(csv_filename, 'w', encoding='UTF-8', newline='') as f:
            dict_writer = csv.DictWriter(f, keys)
            dict_writer.writeheader()
            dict_writer.writerows(self.list_of_forms)

In [6]:
path_to_dir_res = './data/'
data_res = os.listdir(path_to_dir_res)

In [6]:
%%time
path_to_dir = './data_conllu/'
data_conllu = os.listdir(path_to_dir)

result_stat = list()
for i in range(len(data_conllu)):
    
    print(f"{i + 1} из {len(data_conllu)}")
    
    corpus = Corpus('./data_conllu/' + data_conllu[i])
    
    corpus.save_dict_to_json()
    corpus.save_list_to_csv()
    
    stat = corpus.stat
    result_stat.append(stat)    
    
    print(f"Корпус: {stat['corpus_name']}\n")
    print(f"Всего предложений: {stat['count_sent']}")
    print(f"\tРазобрано: {stat['count_parsed_sent']}")
    print(f"\tС ошибками парсинга: {stat['count_error_sent']}\n")
    print(f"Количество найденных примеров глагольного управления в корпусе = {stat['count_vg']}")
    print('Количество уникальных глаголов в корпусе:')
    print(f'\tБез "не": {stat["verb_"]}')
    print(f'\tС "не": {stat["verb_not"]}')
    print('___________________________________________')

1 из 26
Корпус: articles6_2.conllu

Всего предложений: 13608
	Разобрано: 13608
	С ошибками парсинга: 0

Количество найденных примеров глагольного управления в корпусе = 29057
Количество уникальных глаголов в корпусе:
	Без "не": 2416
	С "не": 412
___________________________________________
2 из 26
Корпус: articles_Vestnik_rayona.conllu

Всего предложений: 10412
	Разобрано: 10159
	С ошибками парсинга: 253

Количество найденных примеров глагольного управления в корпусе = 23423
Количество уникальных глаголов в корпусе:
	Без "не": 2236
	С "не": 294
___________________________________________
3 из 26
Корпус: compulenta-2013.txt.conllu

Всего предложений: 472577
	Разобрано: 472577
	С ошибками парсинга: 0

Количество найденных примеров глагольного управления в корпусе = 1330600
Количество уникальных глаголов в корпусе:
	Без "не": 10705
	С "не": 2818
___________________________________________
4 из 26
Корпус: compulenta-2013_short.txt.conllu

Всего предложений: 36199
	Разобрано: 36199
	С ошибка

In [14]:
%%time
path_to_dir = './data_conllu/'
data_conllu = os.listdir(path_to_dir)

result_stat = list()
for i in range(len(data_conllu)):
    corpus_name = data_conllu[i].split('.')[:-1]
    corpus_name = ".".join(corpus_name)
    if f"{corpus_name}.csv" not in data_res or f"{corpus_name}.json" not in data_res:        
        print(f"{i + 1} из {len(data_conllu)}")
    
        corpus = Corpus('./data_conllu/' + data_conllu[i])

        corpus.save_dict_to_json()
        corpus.save_list_to_csv()

        stat = corpus.stat
        result_stat.append(stat)    

        print(f"Корпус: {stat['corpus_name']}\n")
        print(f"Всего предложений: {stat['count_sent']}")
        print(f"\tРазобрано: {stat['count_parsed_sent']}")
        print(f"\tС ошибками парсинга: {stat['count_error_sent']}\n")
        print(f"Количество найденных примеров глагольного управления в корпусе = {stat['count_vg']}")
        print('Количество уникальных глаголов в корпусе:')
        print(f'\tБез "не": {stat["verb_"]}')
        print(f'\tС "не": {stat["verb_not"]}')
        print('___________________________________________')

1 из 29
Корпус: Foreign_Love_Stories.txt.conllu

Всего предложений: 7210321
	Разобрано: 7210321
	С ошибками парсинга: 0

Количество найденных примеров глагольного управления в корпусе = 9707513
Количество уникальных глаголов в корпусе:
	Без "не": 36267
	С "не": 11582
___________________________________________
16 из 29
Корпус: might_and_magic.txt.conllu

Всего предложений: 689996
	Разобрано: 689996
	С ошибками парсинга: 0

Количество найденных примеров глагольного управления в корпусе = 1190847
Количество уникальных глаголов в корпусе:
	Без "не": 13126
	С "не": 4496
___________________________________________
18 из 29
Корпус: nplus1_2020.txt.conllu

Всего предложений: 132881
	Разобрано: 132881
	С ошибками парсинга: 0

Количество найденных примеров глагольного управления в корпусе = 380140
Количество уникальных глаголов в корпусе:
	Без "не": 5521
	С "не": 1262
___________________________________________
19 из 29
Корпус: orcs_and_dwarfs.txt.conllu

Всего предложений: 246028
	Разобрано: 2

In [16]:
df = pd.DataFrame(result_stat)

In [17]:
df.head()

,corpus_name,count_sent,count_parsed_sent,count_error_sent,count_vg,verb_,verb_not
0,Foreign_Love_Stories.txt.conllu,7210321,7210321,0,9707513,36267,11582
1,might_and_magic.txt.conllu,689996,689996,0,1190847,13126,4496
2,nplus1_2020.txt.conllu,132881,132881,0,380140,5521,1262
3,orcs_and_dwarfs.txt.conllu,246028,246028,0,405418,9870,2757
4,profile_ru.txt.conllu,330785,330785,0,843354,11834,2033


In [19]:
df.to_excel("stat_2.xlsx")  

In [22]:
def merge(a, b, path=None):
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])
            else:
                a[key] += b[key]
        else:
            a[key] = b[key]
    return a

In [23]:
def count_stat(a, count=None):
    if count is None: count = 0
    for key in a:
        if isinstance(a[key], dict):
            count = count_stat(a[key], count)
        else:
            count += a[key]
    return count

### Общий csv-файл

In [25]:
path_to_dir = './data/'
data = os.listdir(path_to_dir)

count_stat_per_dict = 0
csv_data_res = None

count = 0

for filename in data:
    if filename[-3:] == "csv":
        
        if csv_data_res is None:
            csv_data_res = pd.read_csv(path_to_dir + filename)  
            count = len(csv_data_res)
        else:
            temp_df = pd.read_csv(path_to_dir + filename) 
            count += len(temp_df)
            
            csv_data_res = pd.concat([csv_data_res, temp_df], ignore_index = True, axis = 0)
    
        print(filename)

Foreign_Love_Stories.txt.csv
articles6_2.csv
articles_Vestnik_rayona.csv
compulenta-2013.txt.csv
detective_for_kidds.txt.csv
detective_masters.txt.csv
dvnovosti.ru_khab.csv
fontanka.txt.csv
gazeta-4-ru-2020.txt.csv
gazeta_3_ru-2018.csv
gorky[ru]_2-2018.csv
habarahabr_2017.txt.csv
ibusines.txt.csv
kosterin72ru.txt.csv
membrana.txt.csv
might_and_magic.txt.csv
naked-science.txt.csv
nplus1_2020.txt.csv
orcs_and_dwarfs.txt.csv
profile_ru.txt.csv
rosbalt_2.csv
russian_love_story.txt.csv
str_fives.txt.csv
syntagrus-deeppavlov.txt.csv
texts_h_100w.txt.csv
texts_m_100w.txt.csv
utro-2014.txt.csv
vogue_2020_2.txt.csv
vzglyad.txt.csv


# Подсчет уникальных сочетаний вида «глагол + предлог + существительное»

In [27]:
len(csv_data_res)

37841081

In [28]:
csv_data_res.head()

,part,verb,adp,noun,case
0,_,соединить,перед,лицом,Ins
1,_,соединить,_,узами,Ins
2,_,слушал,_,слова,Acc
3,_,бросая,_,время,Acc
4,_,бросая,_,взгляд,Acc


In [29]:
csv_data_res.to_csv('./all_data.csv')  

### Общий json-файл

In [30]:
path_to_dir = './data/'
data = os.listdir(path_to_dir)

count_stat_per_dict = 0
json_data_res = None

for filename in data:
    if filename[-4:] == "json":
        
        with open(path_to_dir + filename, 'r', encoding='UTF-8') as f:
            json_data = json.load(f)
            
        count_stat_per_dict += count_stat(json_data)
        
        if json_data_res is None:
            json_data_res = json_data.copy()
        else:
            json_data_res = merge(json_data_res, json_data)  
    
        print(filename)

Foreign_Love_Stories.txt.json
articles6_2.json
articles_Vestnik_rayona.json
compulenta-2013.txt.json
detective_for_kidds.txt.json
detective_masters.txt.json
dvnovosti.ru_khab.json
fontanka.txt.json
gazeta-4-ru-2020.txt.json
gazeta_3_ru-2018.json
gorky[ru]_2-2018.json
habarahabr_2017.txt.json
ibusines.txt.json
kosterin72ru.txt.json
membrana.txt.json
might_and_magic.txt.json
naked-science.txt.json
nplus1_2020.txt.json
orcs_and_dwarfs.txt.json
profile_ru.txt.json
rosbalt_2.json
russian_love_story.txt.json
str_fives.txt.json
syntagrus-deeppavlov.txt.json
texts_h_100w.txt.json
texts_m_100w.txt.json
utro-2014.txt.json
vogue_2020_2.txt.json
vzglyad.txt.json


In [32]:
count_stat(json_data_res)

37841081

In [33]:
count_stat(json_data_res['_'])

35591133

In [34]:
count_stat(json_data_res['не'])

2249948

In [35]:
with open('./all_data.json', 'w', encoding='UTF-8') as f:
    json.dump(json_data_res, f)

# Глаголы

In [36]:
with open('./all_data.json', 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

In [39]:
count_stat(json_data['_']['бежать']['на']['Loc'])

156

In [51]:
b = set()

In [52]:
count = 0
for key in json_data:
    for verb in json_data[key]:
        if count_stat(json_data[key][verb]) == 1:
            b.add(verb)

In [53]:
len(b)

24057

In [55]:
list(b)[:10]

['желайт',
 'искажашие',
 'путовать',
 'пронеслисить',
 'ытоптать',
 'растерзовать',
 'расковаться',
 'истаиалый',
 'заненсный',
 'сцениать']

In [46]:
# 24281 глаголов, которые встретились в корпусе всего один раз. Это могут быть как редкие глаголы, так и глаголы с ошибками

In [47]:
a = set(json_data["_"])

In [48]:
a.update(json_data["не"])

In [49]:
len(a) #Всего уникальных глаголов

66462

In [56]:
len(json_data["_"])

65093

In [57]:
len(json_data["не"])

16781

In [60]:
count = 0
for verb in json_data["не"]:
    if verb not in json_data["_"]:
        count += 1
print(count)

1369


In [61]:
1369 + 49681 + 15412

66462

In [62]:
# считаем статистику по: глагол + предлог + падеж

In [64]:
temp_data = json_data["не"]

In [75]:
res = set()

In [76]:
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                if case != 'Nom':
                    string = f"{verb}_{adp}_{case}"
                    res.add(string)

In [77]:
string

'откупаться_с_Gen'

In [78]:
len(res)

423196

In [82]:
res_2 = set()

In [83]:
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                for noun in json_data[part][verb][adp][case]:
                    if case != 'Nom':
                        string = f"{verb}_{adp}_{case}_{noun}"
                        res_2.add(string)

In [84]:
len(res_2)

6528983

In [85]:
res_3 = set()

In [86]:
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                for noun in json_data[part][verb][adp][case]:
                    res_3.add(noun)

In [87]:
len(res_3)

196128

In [88]:
json_data_res["_"]["бежать"].keys()

dict_keys(['_', 'по', 'в', 'через', 'мимо', 'при', 'у', 'от', 'из', 'с', 'к', 'на', 'впереди', 'под', 'после', 'за', 'без', 'среди', 'о', 'до', 'над', 'из-под', 'вдоль', 'сквозь', 'вокруг', 'посреди', 'позади', 'перед', 'для', 'из-за', 'между', 'меж', 'посередине', 'ради', 'плюс', 'около', 'против', 'спустя', 'вместо', 'порядок', 'накануне', 'помимо', 'вопреки', 'согласно'])

In [94]:
json_data_res["не"]["бежать"]["в"]

{'Acc': {'полиция': 2,
  'подъезд': 1,
  'школа': 1,
  'поисковик': 1,
  'вода': 1,
  'теплица': 1,
  'время': 3,
  'атака': 1,
  'аптека': 1},
 'Loc': {'кроссовок': 1, 'комментарий': 1, 'ступор': 1, 'страх': 1},
 'Gen': {'реклама': 1}}

In [90]:
json_data_res["_"]["бежать"]["через"]

{'Acc': {'деревня': 1,
  'куст': 1,
  'дорога': 7,
  'лабиринт': 1,
  'поле': 5,
  'лес': 8,
  'мгновение': 1,
  'дворик': 1,
  'двор': 4,
  'пара': 2,
  'сад': 1,
  'газон': 1,
  'заросль': 2,
  'снег': 1,
  'люк': 1,
  'лужайка': 3,
  'холл': 4,
  'мыс': 1,
  'граница': 4,
  'библиотека': 1,
  'крыша': 1,
  'дом': 1,
  'долина': 1,
  'цепь': 1,
  'луг': 4,
  'год': 3,
  'путь': 1,
  'кустарник': 1,
  'пустырь': 1,
  'ход': 4,
  'шоссе': 1,
  'поляна': 1,
  'роща': 1,
  'этаж': 1,
  'площадка': 1,
  'ряд': 1,
  'переход': 1,
  'окоп': 1,
  'полянка': 1,
  'стена': 1,
  'дверь': 1,
  'время': 1,
  'час': 1,
  'город': 1,
  'терминал': 1,
  'капилляр': 1,
  'минута': 1,
  'площадь': 1,
  'окно': 1},
 'Gen': {'минута': 4, 'секунда': 2}}

## Считаю статистику по корпусам

In [21]:
class Corpus_2:
    
    def __init__(self, filename):
        self.filename = filename
        self.dict_of_lemmas = dict()
        self.list_of_forms = list()
        self.count_parsed = 0
        self.count_error = 0
        
        self.stat = 0
        
        self.__read_conllu_data__(filename)
        
    def __parse_conllu__(self, text):
        try: 
            result = [tokenlist for tokenlist in conllu.parse(text)]
            return result[0]
        except:
            return None ## сюда можно подставить свой собственный разбор
        
    def __find_verb_government__(self, sent):
        for token in sent:
            if token['upos'] != 'PUNCT':
                self.stat += 1
        
    def __read_conllu_data__(self, filename):
        with open(filename, 'r', encoding='UTF-8') as f:  
            flag = False
            temp_list = list()

            while True: 
                line = f.readline()
                if not line:  # достигли конца файла
                    parse_result = self.__parse_conllu__("".join(temp_list))
                    if parse_result is not None:
                        self.__find_verb_government__(parse_result)
                        self.count_parsed += 1
                    else:
                        self.count_error += 1
                    return 0
                if line[:6] == "# text":
                    if flag is False:  # начинаем считывать новое предложение
                        flag = True
                    else:  # достигли конца разбора предложения
                        flag = False
                        parse_result = self.__parse_conllu__("".join(temp_list))
                        if parse_result is not None:
                            self.__find_verb_government__(parse_result)
                            self.count_parsed += 1
                        else:
                            self.count_error += 1
                        temp_list = list()
                temp_list.append(line)
                
    def __count_verb__(self, part):
        try:
            count = len(self.dict_of_lemmas[part])
        except:
            count = 0
        return count
                
    def __calculate__(self):       
        result_dict = {
            "corpus_name": self.filename.split('/')[-1],
            "count_sent" : self.count_parsed + self.count_error,
            "count_parsed_sent": self.count_parsed, 
            "count_error_sent": self.count_error,
            "count_vg": len(self.list_of_forms), 
            "verb_": self.__count_verb__("_"), 
            "verb_not": self.__count_verb__("не") 
        }
        return result_dict
    
    def __new_filename__(self, ext):
        parts = self.filename.split('conllu')
        new_filename = parts[0][:-1] + parts[1] + ext
        return new_filename
        
    def save_dict_to_json(self, json_filename=None):
        if json_filename is None:
            json_filename = self.__new_filename__('json')
        
        with open(json_filename, 'w', encoding='UTF-8') as f:
            json.dump(self.dict_of_lemmas, f)
    
    def save_list_to_csv(self, csv_filename=None):
        if csv_filename is None:
            csv_filename = self.__new_filename__('csv')
            
        keys = self.list_of_forms[0].keys()
        
        with open(csv_filename, 'w', encoding='UTF-8', newline='') as f:
            dict_writer = csv.DictWriter(f, keys)
            dict_writer.writeheader()
            dict_writer.writerows(self.list_of_forms)

In [6]:
path_to_dir = './data_conllu/'
data_conllu = os.listdir(path_to_dir)

In [17]:
%%time
corpus = Corpus_2('./data_conllu/' + data_conllu[3])

CPU times: user 9min 8s, sys: 7.19 s, total: 9min 16s
Wall time: 9min 32s


In [19]:
corpus.stat

11028843

In [18]:
#8953117 - без пунктуации

In [22]:
%%time
path_to_dir = './data_conllu/'
data_conllu = os.listdir(path_to_dir)

result_stat = dict()
for i in range(len(data_conllu)):
    corpus_name = data_conllu[i].split('.')[:-1]
    corpus_name = ".".join(corpus_name)      
    
    print(f"{i + 1} из {len(data_conllu)}")

    corpus = Corpus_2('./data_conllu/' + data_conllu[i])
    stat = corpus.stat
    result_stat[data_conllu[i]] = stat
    print(data_conllu[i], stat)

1 из 29
Foreign_Love_Stories.txt.conllu 73343266
2 из 29
articles6_2.conllu 193651
3 из 29
articles_Vestnik_rayona.conllu 156367
4 из 29
compulenta-2013.txt.conllu 8953117
5 из 29
detective_for_kidds.txt.conllu 5445601
6 из 29
detective_masters.txt.conllu 12548302
7 из 29
dvnovosti.ru_khab.conllu 192751
8 из 29
fontanka.txt.conllu 16244474
9 из 29
gazeta-4-ru-2020.txt.conllu 9818350
10 из 29
gazeta_3_ru-2018.conllu 4371913
11 из 29
gorky[ru]_2-2018.conllu 1242284
12 из 29
habarahabr_2017.txt.conllu 14924070
13 из 29
ibusines.txt.conllu 5384745
14 из 29
kosterin72ru.txt.conllu 9219897
15 из 29
membrana.txt.conllu 4046765
16 из 29
might_and_magic.txt.conllu 7953042
17 из 29
naked-science.txt.conllu 3621288
18 из 29
nplus1_2020.txt.conllu 2280852
19 из 29
orcs_and_dwarfs.txt.conllu 2590083
20 из 29
profile_ru.txt.conllu 5127225
21 из 29
rosbalt_2.conllu 3603819
22 из 29
russian_love_story.txt.conllu 1764424
23 из 29
str_fives.txt.conllu 451257
24 из 29
syntagrus-deeppavlov.txt.conllu 4497

In [25]:
count = 0
for key in result_stat:
    count += result_stat[key]
print(count)

259634554


In [32]:
per = 0
for key in result_stat:
    temp = round(result_stat[key] / count * 100, 2)
    print(key, temp)
    per += temp

Foreign_Love_Stories.txt.conllu 28.25
articles6_2.conllu 0.07
articles_Vestnik_rayona.conllu 0.06
compulenta-2013.txt.conllu 3.45
detective_for_kidds.txt.conllu 2.1
detective_masters.txt.conllu 4.83
dvnovosti.ru_khab.conllu 0.07
fontanka.txt.conllu 6.26
gazeta-4-ru-2020.txt.conllu 3.78
gazeta_3_ru-2018.conllu 1.68
gorky[ru]_2-2018.conllu 0.48
habarahabr_2017.txt.conllu 5.75
ibusines.txt.conllu 2.07
kosterin72ru.txt.conllu 3.55
membrana.txt.conllu 1.56
might_and_magic.txt.conllu 3.06
naked-science.txt.conllu 1.39
nplus1_2020.txt.conllu 0.88
orcs_and_dwarfs.txt.conllu 1.0
profile_ru.txt.conllu 1.97
rosbalt_2.conllu 1.39
russian_love_story.txt.conllu 0.68
str_fives.txt.conllu 0.17
syntagrus-deeppavlov.txt.conllu 0.17
texts_h_100w.txt.conllu 0.37
texts_m_100w.txt.conllu 0.4
utro-2014.txt.conllu 9.28
vogue_2020_2.txt.conllu 1.25
vzglyad.txt.conllu 14.0


In [23]:
result_stat

{'Foreign_Love_Stories.txt.conllu': 73343266,
 'articles6_2.conllu': 193651,
 'articles_Vestnik_rayona.conllu': 156367,
 'compulenta-2013.txt.conllu': 8953117,
 'detective_for_kidds.txt.conllu': 5445601,
 'detective_masters.txt.conllu': 12548302,
 'dvnovosti.ru_khab.conllu': 192751,
 'fontanka.txt.conllu': 16244474,
 'gazeta-4-ru-2020.txt.conllu': 9818350,
 'gazeta_3_ru-2018.conllu': 4371913,
 'gorky[ru]_2-2018.conllu': 1242284,
 'habarahabr_2017.txt.conllu': 14924070,
 'ibusines.txt.conllu': 5384745,
 'kosterin72ru.txt.conllu': 9219897,
 'membrana.txt.conllu': 4046765,
 'might_and_magic.txt.conllu': 7953042,
 'naked-science.txt.conllu': 3621288,
 'nplus1_2020.txt.conllu': 2280852,
 'orcs_and_dwarfs.txt.conllu': 2590083,
 'profile_ru.txt.conllu': 5127225,
 'rosbalt_2.conllu': 3603819,
 'russian_love_story.txt.conllu': 1764424,
 'str_fives.txt.conllu': 451257,
 'syntagrus-deeppavlov.txt.conllu': 449735,
 'texts_h_100w.txt.conllu': 971519,
 'texts_m_100w.txt.conllu': 1050514,
 'utro-2014